In [1]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from pandas.plotting import table
# import dataframe_image as dfi
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.svm import LinearSVR, SVR
from sklearn.linear_model import SGDRegressor

In [2]:
def clean_bodystyle(x):
    if type(x) == str:
        if x in ['Crossover','Pick-up','Cabriolet','Coupe','-', 'Other']:
            return 'Other'
        else:
            return x

In [3]:
def clean_cooling(x):
    if type(x) == str:
        if x == 'Water-based coolant circulation':
            return x
        else:
            return 'Other'

In [4]:
df = pd.read_csv('output_new_refined_bootstap.csv').drop(['WLTP (km)','NEDC (km)'], axis = 1)

In [5]:
df.dropna(inplace = True)

In [6]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 101)

In [7]:
train_set.drop(['Number of cells', 'Number of modules', 'Voltage (V)', 'Materials'], axis = 1, inplace = True)
test_set.drop(['Number of cells', 'Number of modules', 'Voltage (V)', 'Materials'], axis = 1, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [8]:
train_set.dropna(inplace = True)
test_set.dropna(inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [9]:
train_set.loc[:,'Body style'] = train_set['Body style'].apply(lambda x: clean_bodystyle(x))
test_set.loc[:,'Body style'] = test_set['Body style'].apply(lambda x: clean_bodystyle(x))

train_set.loc[:,'Cooling'] = train_set['Cooling'].apply(lambda x: clean_cooling(x))
test_set.loc[:,'Cooling'] = test_set['Cooling'].apply(lambda x: clean_cooling(x))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [10]:
train_set = pd.get_dummies(data=train_set, columns=['Type of rechargeable battery', 'Cooling','Body style'])
test_set = pd.get_dummies(data=test_set, columns=['Type of rechargeable battery', 'Cooling','Body style'])

In [11]:
train_set.shape

(95, 15)

In [12]:
test_set.shape

(24, 15)

# Correlation

In [13]:
train_set

,Model year,Battery capacity (kWh),Top Speed (km/h),Acceleration from 0 to 100 km/h (s),Curb weight (kg),GVWR (kg),EPA (km),Type of rechargeable battery_Li-Ion Polymer,Type of rechargeable battery_Lithium-Ion,Cooling_Other,Cooling_Water-based coolant circulation,Body style_Hatchback,Body style_Other,Body style_Sedan,Body style_Sport utility vehicle (SUV)
241,2016,90.0,249.930502,3.4,2438.995935,3069.996838,402.013132,0,1,0,1,0,0,0,1
308,2017,35.8,149.990488,9.6,1615.000708,2019.999397,201.167500,0,1,1,0,1,0,0,0
66,2020,64.0,167.049492,7.6,1739.997056,2159.996032,415.209720,1,0,1,0,0,0,0,1
307,2017,35.8,149.990488,9.6,1615.000708,2019.999397,201.167500,0,1,1,0,1,0,0,0
226,2017,100.0,249.930502,3.1,2486.999577,3119.996285,464.938326,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,2017,60.0,209.053266,5.7,2026.998322,2589.996712,337.961400,0,1,0,1,0,0,1,0
232,2016,75.0,224.985732,4.5,2026.998322,2589.996712,400.725660,0,1,0,1,0,0,1,0
243,2016,75.0,209.053266,6.2,2351.996990,2984.998234,383.022920,0,1,0,1,0,0,0,1
73,2020,90.0,200.040962,4.8,2169.997736,2669.996733,376.585560,0,1,0,1,0,0,0,1


In [14]:
corr_matrix = train_set.dropna().corr()
corr_matrix = pd.DataFrame(corr_matrix['EPA (km)'].sort_values(ascending = False))
corr_matrix

,EPA (km)
EPA (km),1.000000
Battery capacity (kWh),0.918576
Top Speed (km/h),0.830613
Cooling_Water-based coolant circulation,0.782999
Curb weight (kg),0.749695
GVWR (kg),0.706556
Body style_Sedan,0.526853
Model year,0.303646
Body style_Sport utility vehicle (SUV),0.287497
Type of rechargeable battery_Lithium-Ion,0.111939


# Standardization

In [15]:
y_train = train_set['EPA (km)']
X_train = train_set.drop('EPA (km)', axis = 1)

y_test = test_set['EPA (km)']
X_test = test_set.drop('EPA (km)', axis = 1)

In [16]:
X_train.columns

Index(['Model year', 'Battery capacity (kWh)', 'Top Speed (km/h)',
       'Acceleration from 0 to 100 km/h (s)', 'Curb weight (kg)', 'GVWR (kg)',
       'Type of rechargeable battery_Li-Ion Polymer',
       'Type of rechargeable battery_Lithium-Ion', 'Cooling_Other',
       'Cooling_Water-based coolant circulation', 'Body style_Hatchback',
       'Body style_Other', 'Body style_Sedan',
       'Body style_Sport utility vehicle (SUV)'],
      dtype='object')

In [17]:
scaler_x = StandardScaler()
scaled_X_train = scaler_x.fit_transform(X_train)
scaled_X_train = pd.DataFrame(scaled_X_train, columns = X_train.columns)

scaler_y = StandardScaler()
y_train = y_train.values.reshape(-1,1)
scaled_y_train = scaler_y.fit_transform(y_train)
#scaled_y_train = pd.DataFrame(scaled_y_train, columns = y_train.columns)

In [18]:
scaled_X_test = scaler_x.transform(X_test)
y_test = y_test.values.reshape(-1,1)
scaled_y_test = scaler_y.transform(y_test)

# Model Training and Testing

## Linear Regression without the standardization

In [19]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [20]:
predictions = lin_reg.predict(X_test)
lin_mse = mean_squared_error(test_set['EPA (km)'], predictions)

In [21]:
np.sqrt(lin_mse)

33.20016732796235

In [22]:
X_train.columns.tolist()

['Model year',
 'Battery capacity (kWh)',
 'Top Speed (km/h)',
 'Acceleration from 0 to 100 km/h (s)',
 'Curb weight (kg)',
 'GVWR (kg)',
 'Type of rechargeable battery_Li-Ion Polymer',
 'Type of rechargeable battery_Lithium-Ion',
 'Cooling_Other',
 'Cooling_Water-based coolant circulation',
 'Body style_Hatchback',
 'Body style_Other',
 'Body style_Sedan',
 'Body style_Sport utility vehicle (SUV)']

In [23]:
lin_reg.coef_[0]

array([ 18.64086045,   3.8013835 ,   0.08369935,  10.23893635,
         0.2214389 ,  -0.19228157,  12.64122566, -12.64122566,
       -28.91752979,  28.91752979,  -5.11755032, -60.24407043,
        51.83380477,  13.52781598])

In [24]:
pd.DataFrame({'feature':X_train.columns.tolist(), 
              'co_eff': lin_reg.coef_[0]})

,feature,co_eff
0,Model year,18.640860
1,Battery capacity (kWh),3.801384
2,Top Speed (km/h),0.083699
3,Acceleration from 0 to 100 km/h (s),10.238936
4,Curb weight (kg),0.221439
5,GVWR (kg),-0.192282
6,Type of rechargeable battery_Li-Ion Polymer,12.641226
7,Type of rechargeable battery_Lithium-Ion,-12.641226
8,Cooling_Other,-28.917530
9,Cooling_Water-based coolant circulation,28.917530


In [25]:
lin_reg.intercept_

array([-37562.4845978])

## Linear Regression

In [26]:
lin_reg = LinearRegression()
lin_reg.fit(scaled_X_train, scaled_y_train)

LinearRegression()

In [27]:
predictions = lin_reg.predict(scaled_X_test)
predictions = scaler_y.inverse_transform(predictions)
lin_mse = mean_squared_error(test_set['EPA (km)'], predictions)
lin_mae = mean_absolute_error(test_set['EPA (km)'], predictions)
lin_mape = mean_absolute_percentage_error(test_set['EPA (km)'], predictions)
lin_r2 = r2_score(test_set['EPA (km)'], predictions)

In [28]:
lin_mse

1102.2511106045304

In [29]:
np.sqrt(lin_mse)

33.20016732795982

In [30]:
lin_mae

23.378969744920287

In [31]:
lin_mape

0.06666037390178703

In [32]:
lin_r2

0.9315420106697032

## Support Vector Regression

In [33]:
svm_reg = LinearSVR(epsilon = 0.5)
svm_reg.fit(scaled_X_train, scaled_y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LinearSVR(epsilon=0.5)

In [34]:
svr_predictions = svm_reg.predict(scaled_X_test)
svr_predictions = scaler_y.inverse_transform(svr_predictions)
svr_mse = mean_squared_error(test_set['EPA (km)'], svr_predictions)
svr_mae = mean_absolute_error(test_set['EPA (km)'], svr_predictions)
svr_mape = mean_absolute_percentage_error(test_set['EPA (km)'], svr_predictions)
svr_r2 = r2_score(test_set['EPA (km)'], svr_predictions)

In [35]:
np.sqrt(svr_mse)

40.134724706870564

In [36]:
svr_mae

32.88802136922167

In [37]:
svr_mape

0.09856276752089736

In [38]:
svr_r2

0.8999575840433911

## Ridge Regression

In [40]:
ridge_reg = Ridge(alpha=1, solver = "cholesky")
ridge_reg.fit(scaled_X_train, scaled_y_train)

Ridge(alpha=1, solver='cholesky')

In [41]:
ridge_predictions = ridge_reg.predict(scaled_X_test)
ridge_predictions = scaler_y.inverse_transform(ridge_predictions)
ridge_mse = mean_squared_error(test_set['EPA (km)'], ridge_predictions)
ridge_mae = mean_absolute_error(test_set['EPA (km)'], ridge_predictions)
ridge_mape = mean_absolute_percentage_error(test_set['EPA (km)'], ridge_predictions)
ridge_r2 = r2_score(test_set['EPA (km)'], ridge_predictions)

In [42]:
np.sqrt(ridge_mse)

33.54022225359805

In [75]:
ridge_mae

15.54227577434836

In [76]:
ridge_mape 

0.07155859713186179

In [77]:
ridge_r2

0.9301239307411804

## Lasso Regression

In [43]:
lasso_reg = Ridge(alpha=0.1)
lasso_reg.fit(scaled_X_train, scaled_y_train)

Ridge(alpha=0.1)

In [44]:
lasso_predictions = lasso_reg.predict(scaled_X_test)
lasso_predictions = scaler_y.inverse_transform(lasso_predictions)
lasso_mse = mean_squared_error(test_set['EPA (km)'], lasso_predictions)
lasso_mae = mean_absolute_error(test_set['EPA (km)'], lasso_predictions)
lasso_mape = mean_absolute_percentage_error(test_set['EPA (km)'], lasso_predictions)
lasso_r2 = r2_score(test_set['EPA (km)'], lasso_predictions)

In [45]:
np.sqrt(lasso_mse)

33.08404302848651

In [46]:
lasso_mae

23.57040736693983

In [47]:
lasso_mape

0.06720925872525914

In [83]:
lasso_r2

0.9320118154827979

## Elastic Net

In [49]:
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(scaled_X_train, scaled_y_train)

ElasticNet(alpha=0.1)

In [50]:
elastic_predictions = elastic_net.predict(scaled_X_test)
elastic_predictions = scaler_y.inverse_transform(elastic_predictions)
elastic_mse = mean_squared_error(test_set['EPA (km)'], elastic_predictions)
elastic_mae = mean_absolute_error(test_set['EPA (km)'], elastic_predictions)
elastic_mape = mean_absolute_percentage_error(test_set['EPA (km)'], elastic_predictions)
elastic_r2 = r2_score(test_set['EPA (km)'], elastic_predictions)

In [51]:
np.sqrt(elastic_mse)

37.42277870799179

In [52]:
elastic_mae

29.674269110092624

In [53]:
elastic_mape

0.08799001139476297

In [54]:
elastic_r2

0.9130207502462218

## Non-linear SVR

In [63]:
svm_poly_reg = SVR(kernel = 'linear', degree = 1, C=100, epsilon = 0.1)
svm_poly_reg.fit(scaled_X_train, scaled_y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVR(C=100, degree=1, kernel='linear')

In [64]:
svr_poly_predictions = svm_poly_reg.predict(scaled_X_test)
svr_poly_predictions = scaler_y.inverse_transform(svr_poly_predictions)
svr_poly_mse = mean_squared_error(test_set['EPA (km)'], svr_poly_predictions)
svr_poly_mae = mean_absolute_error(test_set['EPA (km)'], svr_poly_predictions)
svr_poly_mape = mean_absolute_percentage_error(test_set['EPA (km)'], svr_poly_predictions)
svr_poly_r2 = r2_score(test_set['EPA (km)'], svr_poly_predictions)

In [65]:
np.sqrt(svr_poly_mse)

32.131780202566105

In [58]:
svr_poly_mae

22.36695488586879

In [59]:
svr_poly_mape

0.06173999292134034

In [60]:
svr_poly_r2

0.9386551924777684

# Further Analysis

In [61]:
test_set['EPA (km)'].mean()

364.5421237321719

In [62]:
avg_range = test_set['EPA (km)'].mean()
np.sqrt(svr_poly_mse)/avg_range

0.08621233098343364